# Analyse d’un modèle avec demandes, stockage hydroélectrique et gaz

### Import des librairies utilisées pour résoudre les problèmes

In [85]:
import numpy as np
from fileData import FileData
import matplotlib.pyplot as plt
import cvxpy

### Variables communes

In [86]:
# Couts d'installation amortis des éoliennes
cout_instal_onshore  = 168903 # Coût *amorti sur un an* pour l'installation d'un MW éolien on-shore (euros/MW/an)
cout_instal_offshore = 300336 # Coût *amorti sur un an* pour l'installation d'un MW éolien on-shore (euros/MW/an)

# Couts d'installation amortis et coûts de fonctionnement des centrales au gaz
cout_instal_gaz = 94956 # Coût *amorti sur un an* pour l'installation d'un MW de centrale au gaz (euros/MW/an)
cout_fonct_gaz  = 65    # Coût de fonctionnement pour la production d'un MWh par centrale au gaz (euros/MWh)

# Stockage disponible par pays (en MWh)
stockage = [0.3*1e6 , 3.2*1e6 , 0.01*1e6 , 0 , 18.4*1e6 , 9.8*1e6 , 0.24*1e6 , 7.9*1e6 , 0.005*1e6 , 84.147*1e6 , 0 , 2.6*1e6 , 1.2*1e6 , 33.756*1e6 , 8.4*1e6]

# Puissances maximales de turbinage et de pompage disponibles par pays (en MW)
turbinage = [8587 , 12009 , 1417 , 9 , 18372 , 25132 , 527 , 21117 , 1140 , 28941 , 37 , 5052 , 4269 , 16637 , 15101]
pompage   = [5223 , 3580 , 1307 , 0 , 5347 , 4303 , 292 , 7544 , 1100 , 1396 , 0 , 1029 , 2744 , 45 , 1636 ]

# Rendement pour le turbinage (sans unité)
heta = 0.85

# Longueur de la période utilisée
T = 3

### Extraction des données partagées dans les modèles

In [87]:
data = FileData()

# Matrice de la consommation agrégée
D = np.sum(data.getConsoData(), axis=0)
# Rendement des éoliennes onshore et offshore
R = data.getRend()
# Capacités électriques éolien
c = data.getCapacities()



### Fonctions communes
Fonctions communes à tous les modèles

In [88]:
def periodisation(A, T):
    """
    Additionne toutes les valeurs de A dans une même période.
    input A : Matrice à périodiser
    input T : Longueur de la période
    output : Matrice A périodisée
    """
    # p : nombre de période
    # r : reste du nombre de période
    p = A.shape[1] // T
    r = A.shape[1] % T
    # On coupe la matrice car sinon il n'y a pas assez de données de remplir la dernière période
    A = A[:,:-r] if r!=0 else A
    # C : matrice pour additionner les productions d'une période
    C = np.zeros((A.shape[1], p))
    for i in range(p):
        C[i * T:(i + 1) * T, i] = 1
    # P : matrice périodisée
    P = np.dot(A, C)
    return P

def E_eolienne(R, c, T):
    A = np.zeros(R.shape)
    for i in range(len(c)):
        A[i] = c[i] * R[i]
        
    A = periodisation(A, T)
    return A

### Produits des fonctions communes
Variables communes nécessitant un calcul

In [89]:
# Périodisation de D
D = periodisation(np.array([D]), T)
# E : matrice de la production éolienne par site par période
E = E_eolienne(R, c, T)
# e : vecteur de production par période
e = np.sum(E,axis=0)
# s : scalaire, stockage total hydroélectrique
s = np.sum(stockage)
# pPompage : scalaire, puissance de pompage totale
pPompage = np.sum(pompage)
# pTurbinage : scalaire, puissance de turbinage totale
pTurbinage = np.sum(turbinage)
# wApport : Débit naturel de remplissage du stockage
wApport = np.sum(data.getHydroData())

411320506.21439993


## Modèle 4
On considère d’abord un modèle européen agrégé avec installation d’éoliennes et utilisation des capacités hydroélectriques, mais pas de centrales au gaz. Ce modèle agrégé ne possède donc qu’une seule entité pour la production, le stockage et la consommation d’électricité (additionnez les capacités hydroélectriques et les demandes de tous les pays qui la composent).
On chercher à calculer la valeur minimale du prix moyen de l’électricité consommée.

## Modèle 5
Considérons le modèle 4 avec la contrainte supplémentaire suivante : sur chaque site  éolien potentiel, il faut choisir entre installer 100%, 50% ou 0% de la capacité maximale installable.

In [90]:
# TODO

## Modèle 6
Revenons au modèle initial de la question 4 et incorporons la possibilité d’installer et d’utiliser des centrales au gaz.

In [91]:
# TODO